Antes de empezar: un poco de documentación para enterder ligeramente cómo funciona HTML:

https://www.w3schools.com/html/html_intro.asp

# Web Scraping: Selenium

En este notebook veremos cómo automatizar la extracción de datos de una página web simulando la navegación. Para ello utilizaremos la librería Selenium:
https://selenium-python.readthedocs.io/

El software libre Selenium es un **framework** para realizar test automatizados de software a aplicaciones web. En principio, fue desarrollado para poner a prueba páginas y apps web, pero el **WebDriver** de Selenium también puede usarse con Python para realizar scraping. Si bien Selenium en sí no está escrito en Python, con este lenguaje de programación es posible acceder a las funciones del software.

A diferencia de Scrapy y de BeautifulSoup, Selenium no trabaja con el texto fuente en HTML de la web en cuestión, sino que carga la página en un navegador sin interfaz de usuario. El navegador interpreta entonces el código fuente de la página y crea, a partir de él, un **Document Object Model** (modelo de objetos de documento o DOM). Esta interfaz estandarizada permite poner a prueba las interacciones de los usuarios. De esta forma se consigue, por ejemplo, simular clics y rellenar formularios automáticamente. Los cambios en la web que resultan de dichas acciones se reflejan en el DOM. La estructura del proceso de web scraping con Selenium es la siguiente:

URL → Solicitud HTTP → HTML → Selenium → DOM

In [4]:
# Imports
import pandas as pd
from selenium import webdriver

Deberemos descargar un archivo Chrome driver para selenium en https://chromedriver.chromium.org/downloads, es importante que compruebes la versión de Chrome que utilizas y descargaues el driver correspondiente.
Una vez descargado crearemos una variable con la dirección completa del path al chromedriver 

El objeto driver es con el que trabajaremos a partir de ahora

In [10]:
# especificamos el path hasta nuestro driver recién descargado:
chrome_driver_path = 'C:\\Users\\ggarr\\OneDrive\\Escritorio\\chromedriver.exe'
options = webdriver.ChromeOptions()
# option.add_argument('headless')
# headless nos permite no tener que ver la navegación en la ventana de Chrome

In [11]:
# Creamos el driver con el que nos vamos a manejar en la sesión de scrapeo:
driver = webdriver.Chrome(executable_path = chrome_driver_path, options = options)

In [12]:
# indicamos la URL de la página web a la que queremos acceder:
url = 'https://www.filmaffinity.com/es/main.html'
driver.get(url)

# el objeto driver nos va a permitir alterar el estado del la página


¡Vaya! Nos hemos encontrado con un pop-up que nos pide aceptar cookies o algo po el estilo, si nos lo queremos saltar, para poder automatizar el proceso deberemos:

1. Buscamos el botón
2. Hacemos click en el botón

Selenium nos permite buscar elementos por etiqueteas de html: https://selenium-python.readthedocs.io/locating-elements.html

CUIDADO! Los metodos de busqueda de elementos estan "duplicados". Tenemos *find_element_by_tag_name* y *find_elements_by_tag_name* el primero nos devolverá el primer elemento que encuentre (aunque haya más), el segundo nos devolverá una lista con todos los elementos que encuentre (aunque solo sea uno)

In [13]:
# Navegando en filmaffinity, en el pop-up:
elements_by_tag = driver.find_elements_by_tag_name('button')
elements_by_tag

elements_by_xpath = driver.find_element_by_xpath('/html/body/div[1]'/div/div)

Una vez tenemos los elementos podemos hacer varias cosas con ellos

Podemos extraer todos los atributos que tenga

In [ ]:

# obtenemos todos sus métodos y atributos:

dir(elements_by_tag)


Podemos evaluar que tipo de elemento es (tag)

In [ ]:
element_by_tag.size

In [ ]:
element_by_tag.get_attribute('size')

Podemos sacar el valor que tiene (el texto)

In [ ]:
element_by_xpath.text

In [ ]:
for i in range (0,len(elements_by_tag)):
    print (elements_by_tag[i].text)


Incluso podemos guardar una imagen del elemento

In [ ]:
# Vemos que es tipo 'WebElement' y en la documentación podremos encontrar sus métodos

type(element_by_xpath)




In [ ]:
# guardamos como 'mi_imagen.png' la imagen asociada al xpath
element_by_xpath.screenhot('mi_imagen.png')

Evaluamos que elementos hemos encontrado por el tag:

In [ ]:
for index, element in enumerate('elements_by_tag'):
    print('Elemento:', index)
    print('Texto del elemento', index, 'es', element.text)
    print ('El tag del elemento', index, 'es', element.tag_name)
    element.screenshot('mi_imagen' +str(index)+'.png')

Nos quedamos con el botón que nos interesa

In [ ]:
boton_aceptar = elements_by_tag[2]


Si el elemento es interactivo podremos hacer más cosas además de las anteriores. Por ejemplo: hacer click

In [ ]:
boton_aceptar.click()

Buscamos una película por título

In [ ]:
from selenium.webdriver.common.keys import Keys

buscador = driver.find

In [ ]:
buscador = driver.find_element_by_xpath('html/body/div[2]/div[1]/div/div[2]/form/input[1]')

In [ ]:
buscador.send_keys('interstellar')

In [ ]:
# una vez escrita la búsqueda deberíamos poder activarla:
buscador.send_keys(keys.ENTER)

In [ ]:
# volvemos a la página anterior
driver.back()

### Vamos a buscar todas las películas que se estrenan el próximo viernes

1. Cogemos los containers que hay en la zona central

In [ ]:
menu_lateral = driver.find_element_by_id('lsmenu')
menu_lateral

In [ ]:
secciones_menu = menu_lateral.find_elements_by_tag_name('a')

2. Vemos con cuál nos tenemos que quedar

In [ ]:
for a in secciones _menu:
    if a.text == 'Próximos estrenos':
        a.click()
        break

Accedemos al container central, en el que aparecen los estrenos por semana que queremos ver, exactamente igual que hemos hecho antes

In [ ]:
cajon_central = driver.find_elements_by_id('main-wrapper-rdcat')

In [ ]:
type(cajon_central)

In [ ]:
for semana in cajon_central:
    print (semana.find_element_by_tag_name('div').text)
    print(semana.find_element_by_tag_name('div').get_attibute('id'))

Buscamos cómo acceder a las películas

In [ ]:
for semana in cajon_central:
    fecha = semana.find_element_by_tag_name('div').get_attribute('id')
    if fecha == '2021-05-21':
        break


In [ ]:
caratula = semana.find_elements_by_class_name('mc-poster')
# iterar para obtener los links de las peliculas de estreno

lista_pelis = {}
for peli in caratulas:
    lista_pelis.append(peli.find_element_by_tag_name('a').get_attribute('href'))

In [ ]:
len(lista_pelis)

Una vez tenemos todas las urls vamos a ver qué hacemos con cada una de ellas

In [ ]:
# Accedemos a la página de la primera pelicula
driver.get(lista_pelis[0])


In [ ]:
driver.back()

Vamos a ver el proceso que deberíamos hacer con cada una de las películas:

1. Sacamos toda la información que nos interesa

In [ ]:
# titulo, nota, numero de votos y ficha técnica
titulo = driver.find_element_by_xpaths('').text
nota = driver.find_element_by_xpaths('').text
votos = driver.find_element_by_xpaths('').text
ficha = driver.find_element_by_xpaths('').text


2. Creamos una lista a partir de la ficha técnica

In [ ]:
# Los nombres estan con tag  = 'dt' y los valores con 'dd'
ficha_claves = []
ficha_valores = []

for name in ficha.find_elements_y_tag_name('dt'):
    ficha_claves.append(name.text)
for value in ficha.find_elements_y_tag_name('dd'):   
    ficha_valores.append(value.text)

3. Creamos un dataframe con la info

In [ ]:
columns = ['Titulo', 'Nota', 'Votos']
columns.extend[ficha_claves]

In [ ]:
values = [titulo, nota, votos]
values.extend[ficha_valores]

In [ ]:
pd.DataFrame([valores], columns = columns)

Ahora vamos a crear una función que nos haga todo esto para cada una de las películas:

In [ ]:
def sacar_info():
    titulo = driver.find_element_by_xpaths('').text

    try:
        nota = driver.find_element_by_xpath('/       ')
        votos = driver.find_element_by_xpath('/       ')

    except:
        nota = None
        votos= None

    ficha = driver.find_element_by_xpath('/        ')
    return titulo, nota, votos, icha

def sacar_ficha():
    ficha_claves = []
    ficha_valores = []

    for name in ficha.find_elements_y_tag_name('dt'):
        ficha_claves.append(name.text)
    for value in ficha.find_elements_y_tag_name('dd'):   
        ficha_valores.append(value.text)

    return ficha_claves, ficha_valores

def montar_df(ficha_claves, ficha_valores, titulo, nota, votos):
    columns = ['Titulo', 'Nota', 'Votos']
    columns.extend[ficha_claves]
    values = [titulo, nota, votos]
    values.extend[ficha_valores]

    return pd.DataFrame([valores], columns = columns)

def nueva_peli():
    titulo, nota, votos, ficha = sacar_info(driver)
    ficha_claves = ficha_valores = sacar_ficha(ficha)
    df_peli = mntar_df(ficha_claves, ficha_valores, titulo, nota, votos)
    
    return df_peli
    
    
    


## Modo Dios: moviendonos entre ventanas

Vamos a ver cómo nos podemos mover entre ventanas del navegador

Abrir nueva ventana:

In [ ]:
driver.execute_script('window.open('');')

Movernos a otra ventana

In [ ]:
driver.switch_to.window(driver.window_handles[0])

Cerrar ventana

In [ ]:
driver.close()

Una vez cerramos la ventana tenemos que indicarle a qué ventana tiene que ir

Sabiendo cómo podemos movernos por entre las ventanas y sabiendo cómo extraer de cada página toda la información que necesitamos vamos a crear nuestro dataframe:

In [ ]:
# para abrir todos los links en lista_pelis

for link in lista_pelis:
    driver.execute_script('window.open("'+link+'");')
    driver.switch_to.window(driver.window_handles[-1])
    driver.get(link)



In [ ]:
# Creamos un dataframe con todas las pelis que se estrenan la próxima semana:

df_peliculas = pd.DataFrame()

for link in lista_pelis:
    driver.execute_script('window.open("'+link+'");')
    driver.switch_to.window(driver.window_handles[-1])
    driver.get(link)
    nueva_pelicula = nueva_peli(driver)
    df_peliculas = df_peliculas.append(nueva_pelicula)


¡Tachán! Ya tenemos un dataframe con todas las películas que se van a estrenar el próximo viernes